
Why was memset so bad?
How to rethink my approach
Make better debug equipment

Move towards the firmware code.
What can I do to be more pragmatic / ergnonomic. This hardo mode is not that great
Easier no aliasing


The amount of harness per program isn't great.

How to get the wheel back turning

Get started on OS / rtos something?

spacer

Look at old posts with ideas

separation logic something something
learn more dafny / verus

hol-light examples

interrupts and interrupt invariants
cbmc as a backend
lean as a backend

wasm
6502
nand2tetris

rpython interpreter of pcode

device modelling






6502
https://verso.uidaho.edu/esploro/outputs/graduate/Formal-Verification-for-Assembly-Language-Operational/996697046701851
C parser

https://www.philipzucker.com/asm_verify4/

https://github.com/awslabs/s2n-bignum/blob/main/x86/tutorial/simple.ml

icicle

https://www.youtube.com/watch?v=zeJ_V0NRIk0 ml-kem hans becker. connect to cbmc?

prettier printing of goals

riscv formal comparison  https://github.com/YosysHQ/riscv-formal/blob/main/insns/insn_add.v

simd
spacer

wasm https://github.com/nneonneo/ghidra-wasm-plugin/releases/tag/v2.3.2
gameboy https://github.com/Gekkio/GhidraBoy

aexpr interpreter
memset

remove ram32

relational properties

https://github.com/fraunhoferfokus/acsl-by-example/blob/master/ACSL-by-Example.pdf


In [ ]:
def MayChange(mem1 : smt.ArrayRef,mem2 : smt.ArrayRef, addrs : list[int | smt.BitVecRef]) -> smt.BoolRef:
    """
    forall addr not in addr, mem1[addr] == mem2[addr]
    Can instead be done by masking out the addresses into a quantifier free statement.
    """
    for a in addrs:
        mem1 = bv.store_concat(mem1, addr, 0)
        mem2 = bv.store_concat(mem2, addr, 0)
    return mem1 == mem2

# smt ir



TypeError: tuple expected at most 1 argument, got 3

# lean print



In [ ]:
import pypcode
import kdrag.contrib.pcode as pcode


def lean_varnode(vn: pypcode.Varnode) -> str:
    return f"({vn.space.name()} {vn.offset:x} {vn.size})"

def lean_insn(insn: pypcode.PcodeOp) -> str:
    args = " ".join(lean_varnode(vn) for vn in insn.inputs)
    if insn.output is not None:
        return f"({insn.code.name().lower()} {lean_varnode(insn.output)} {args})"
    else:
        return f"({insn.code.name().lower()} {args})"




# cut diagrams
cut as a diagram
```mermaid
flowchart TD
   A
   B
   C
   D
   E
   F
   G
   A --> B
   B --> C
   B --> E
   E --> D
   D --> F 
   F --> C
   F --> B
   E --> G
```

```mermaid
flowchart TD
   A
   Benter
   Bexit[/Bexit\]
   C
   D
   E
   F
   G
   A --> Benter
   Bexit --> C
   Bexit --> E
   E --> D
   D --> F 
   F --> C
   F --> Benter
   E --> G
   Benter[\Benter/] -.cut.-x Bexit
```

```mermaid
flowchart LR
subgraph original
   direction TB
   A --> B
   B --> C
   B --> E
   E --> D
   D --> F 
   F --> C
   F --> B
   E --> G
end
subgraph processed
   direction TB
   A2["A"]
   Benter
   Bexit[/Bexit\]
   C2["C"]
   D2["D"]
   E2["E"]
   F2["F"]
   G2["G"]
   A2 --> Benter
   Bexit --> C2
   Bexit --> E2
   E2 --> D2
   D2 --> F2 
   F2 --> C2
   F2 --> Benter
   E2 --> G2
   Benter[\Benter/] -.cut.-x Bexit
end
original ==INSERT CUT==> processed
```

# memset


In [1]:
%%file /tmp/mycpy.s
    .text
    .globl mycpy
    .align 2

# a0 is src
# a1 is dst
# a2 is len
mycpy:
    beq a2, x0, .done
.loop:
    lb t0, 0(a0)
    sb t0, 0(a1)
    addi a0, a0, 1
    addi a1, a1, 1
    addi a2, a2, -1
    bne a2, x0, .loop
.done:
    ret

Writing /tmp/mycpy.s


In [2]:
%%file /tmp/test.c
#include <stdint.h>
#include <stdio.h>
extern void mycpy(const char* src, char* dst, size_t len);
int main() {
    int32_t src[] = {1, 2, 3, 4, 5};
    int32_t dst[5] = {0};
    mycpy((const char*)src, (char*)dst, sizeof(src));
    for (int i = 0; i < 5; i++) {
        if (dst[i] != src[i]) {
            printf("Error at index %d: expected %d, got %d\n", i, src[i], dst[i]);
            return 1;
        }
    }
}

Writing /tmp/test.c


In [3]:
!nix-shell -p pkgsCross.riscv32.buildPackages.gcc --run "riscv32-unknown-linux-gnu-gcc -O2 /tmp/mycpy.s /tmp/test.c  -o /tmp/mycpy" && qemu-riscv32 /tmp/mycpy && echo "Success"

Success


In [4]:
import kdrag.contrib.pcode as pcode
import kdrag.contrib.pcode.asmspec as asmspec
from kdrag.all import *
import kdrag.theories.bitvec as bv
BV32 = smt.BitVecSort(32)
BV8 = smt.BitVecSort(8)

ctx = pcode.BinaryContext("/tmp/mycpy", langid="RISCV:LE:32:default")
a0 = ctx.state_vars["a0"]
a1 = ctx.state_vars["a1"]
a2 = ctx.state_vars["a2"]
ram = ctx.state_vars["ram"]
#write = ctx.state_vars["write"]

spec = asmspec.AsmSpec(ctx)
src,dst,N = smt.Consts("src dst N", BV32)
spec.add_entry("mycpy", spec.find_label("mycpy"), smt.And(
    a0 == src,
    a1 == dst,
    #smt.Distinct(a0, a0 + 1, a0-1, a1),
    smt.ULT(src,1024),
    smt.ULT(dst, 5012),
    smt.UGE(dst, 2000),
    a2 == N,
    N == 2,
    #smt.ULT(N, 2),
    ))
i = smt.Const("i", BV32)
spec.add_assert(".loop", spec.find_label(".loop"), smt.And(
    src + N == a0 + a2,
    dst + N == a1 + a2,
    #smt.ForAll([i], smt.ULT(0,i), smt.ULT(i, N-a2), ram[src + i] == ram[dst + i])
    ))

spec.add_exit(".done", spec.find_label(".done"), smt.And(ram[src] == ram[dst], ram[src+N-1] == ram[dst+N-1]))

ps = asmspec.AsmProofState(spec)
for i in range(len(ps.vcs)):
    ps.auto(i)
ps
#ps
#l = ps.lemma(0)
#l.intros()


Unknown reloc 3 on RISCV:LE:32:RV32GC
Unknown reloc 1 on RISCV:LE:32:RV32GC
Unknown reloc 5 on RISCV:LE:32:RV32GC
Unknown reloc 10 on RISCV:LE:32:RV32GC
Unknown reloc 58 on RISCV:LE:32:RV32GC


Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 0) PCODE IMARK ram[400650:2]
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 1) PCODE unique[10800:1] = a2 == 0x0
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 2) PCODE if (unique[10800:1]) goto ram[400662:4]
Executing SpecStmt: Assert(.loop, 0x400652, And(src + N == a0 + a2, dst + N == a1 + a2))
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 0) PCODE IMARK ram[400652:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 1) PCODE unique[580:4] = 0x0
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 2) PCODE unique[2880:4] = a0 + unique[580:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 3) PCODE unique[2900:1] = *[ram]unique[2880:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 4) PCODE t0 = sext(unique[2900:1])
Executing 0x400656/4: sb t0,0x0(a1) at (4195926, 0) PCODE IMARK ram[400656:4]
Executing 0x400656/4: sb t0,0x0(a1) at (4195926, 1) PCODE unique[600:4] = 0x0
Executing 0x400656/4: sb t0,0x0(a1) at (4195926,

AsmProofState(num_vcs=3, num_pfs=3)

In [54]:
i = smt.Const("i", BV32)
checkeq = smt.Function("checkeq", smt.ArraySort(BV32,BV8), BV32, smt.BoolSort())
checkeq = kd.define("checkeq", [ram, i], 
    smt.If(i == 0,
        True,
        smt.And(
        ram[src + i - 1] == ram[dst+i - 1],
        checkeq(ram, i-1)))
)

       checkeq(ram, i) ==
       If(i == 0,
          True,
          And(ram[src + i] == ram[dst + i],
              checkeq(ram, i - 1)))) to ForAll([ram, i],
       checkeq(ram, i) ==
       If(i == 0,
          True,
          And(ram[src + i - 1] == ram[dst + i - 1],
              checkeq(ram, i - 1))))


In [59]:
import kdrag.contrib.pcode as pcode
import kdrag.contrib.pcode.asmspec as asmspec
from kdrag.all import *
import kdrag.theories.bitvec as bv
BV32 = smt.BitVecSort(32)
BV8 = smt.BitVecSort(8)

ctx = pcode.BinaryContext("/tmp/mycpy", langid="RISCV:LE:32:default")
a0 = ctx.state_vars["a0"]
a1 = ctx.state_vars["a1"]
a2 = ctx.state_vars["a2"]
ram = ctx.state_vars["ram"]
#write = ctx.state_vars["write"]

spec = asmspec.AsmSpec(ctx)
src,dst,N = smt.Consts("src dst N", BV32)
spec.add_entry("mycpy", spec.find_label("mycpy"), smt.And(
    a0 == src,
    a1 == dst,
    #smt.Distinct(a0, a0 + 1, a0-1, a1),
    smt.ULT(src,1024),
    smt.ULT(dst, 5012),
    smt.UGE(dst, 2000),
    a2 == N,
    N == 2,
    #smt.ULT(N, 2),
    ))
i = smt.Const("i", BV32)
spec.add_assert(".loop", spec.find_label(".loop"), smt.And(
    src + N == a0 + a2,
    dst + N == a1 + a2,
    checkeq(ram, N - a2)
    #smt.ForAll([i], smt.ULT(0,i), smt.ULT(i, N-a2), ram[src + i] == ram[dst + i])
    ))

spec.add_exit(".done", spec.find_label(".done"), checkeq(ram, N))

ps = asmspec.AsmProofState(spec)
for i in range(len(ps.vcs)):
    print(i)
    ps.auto(i, by=[checkeq.defn])
ps

Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 0) PCODE IMARK ram[400650:2]
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 1) PCODE unique[10800:1] = a2 == 0x0
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 2) PCODE if (unique[10800:1]) goto ram[400662:4]
Executing SpecStmt: Assert(.loop, 0x400652, And(src + N == a0 + a2,
    dst + N == a1 + a2,
    checkeq(ram, N - a2)))
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 0) PCODE IMARK ram[400652:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 1) PCODE unique[580:4] = 0x0
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 2) PCODE unique[2880:4] = a0 + unique[580:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 3) PCODE unique[2900:1] = *[ram]unique[2880:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 4) PCODE t0 = sext(unique[2900:1])
Executing 0x400656/4: sb t0,0x0(a1) at (4195926, 0) PCODE IMARK ram[400656:4]
Executing 0x400656/4: sb t0,0x0(a1) at (4195926, 1) PCODE unique[600:4] = 0x0
Executing 0x400656

AsmProofState(num_vcs=3, num_pfs=3)

In [61]:
import kdrag.contrib.pcode as pcode
import kdrag.contrib.pcode.asmspec as asmspec
from kdrag.all import *
import kdrag.theories.bitvec as bv
BV32 = smt.BitVecSort(32)
BV8 = smt.BitVecSort(8)

ctx = pcode.BinaryContext("/tmp/mycpy", langid="RISCV:LE:32:default")
a0 = ctx.state_vars["a0"]
a1 = ctx.state_vars["a1"]
a2 = ctx.state_vars["a2"]
ram = ctx.state_vars["ram"]
#write = ctx.state_vars["write"]

spec = asmspec.AsmSpec(ctx)
src,dst,N = smt.Consts("src dst N", BV32)
Nc = 2
spec.add_entry("mycpy", spec.find_label("mycpy"), smt.And(
    a0 == src,
    a1 == dst,
    #smt.Distinct(a0, a0 + 1, a0-1, a1),
    smt.ULT(src,1024),
    smt.ULT(dst, 5012),
    smt.UGE(dst, 2000),
    a2 == N,
    N == Nc,
    #smt.ULT(N, 2),
    ))
i = smt.Const("i", BV32)
spec.add_cut(".loop", spec.find_label(".loop"), smt.And(
    src + N == a0 + a2,
    dst + N == a1 + a2,
    smt.ULT(src,1024),
    smt.ULT(dst, 5012),
    smt.UGE(dst, 2000),
    checkeq(ram, N - a2),
    N == Nc
    #smt.ForAll([i], smt.ULT(0,i), smt.ULT(i, N-a2), ram[src + i] == ram[dst + i])
    ))

spec.add_exit(".done", spec.find_label(".done"), checkeq(ram, N))

ps = asmspec.AsmProofState(spec)
for i in range(len(ps.vcs)):
    print(i)
    ps.auto(i, by=[checkeq.defn])

Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 0) PCODE IMARK ram[400652:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 1) PCODE unique[580:4] = 0x0
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 2) PCODE unique[2880:4] = a0 + unique[580:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 3) PCODE unique[2900:1] = *[ram]unique[2880:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 4) PCODE t0 = sext(unique[2900:1])
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 0) PCODE IMARK ram[400650:2]
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 1) PCODE unique[10800:1] = a2 == 0x0
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 2) PCODE if (unique[10800:1]) goto ram[400662:4]
Executing SpecStmt: Cut(.loop, 0x400652, And(src + N == a0 + a2,
    dst + N == a1 + a2,
    ULT(src, 1024),
    ULT(dst, 5012),
    UGE(dst, 2000),
    checkeq(ram, N - a2),
    N == 2))
Executing 0x400656/4: sb t0,0x0(a1) at (4195926, 0) PCODE IMARK ram[400656:4]
Executing 0x400656/4: sb t0

IndexError: tuple index out of range

In [103]:
l = ps.lemma(2)
l.intros()
#l.emt()
l.unfold(checkeq)
l.emt()
l.cases(l.top_goal().goal.arg(0))
l.auto()
l.emt()
l.split()
#for _ in range(3):
l.clear(0)
l.clear(0)
l
#l.auto(by=[ctx.select32.defn])
l.auto(by=[ctx.select32.defn, ctx.store32.defn])
#l.auto(by=[ctx.select32.defn, ctx.store32.defn, checkeq.defn])
l.unfold(ctx.select32, ctx.store32)
l.simp()

?|= (let ((a!1 (= (bvadd src N)
              (bvadd (select32le (register state0) &a0)
                     (select32le (register state0) &a2))))
      (a!2 (= (bvadd dst N)
              (bvadd (select32le (register state0) &a1)
                     (select32le (register state0) &a2))))
      (a!3 (or (not (= ((_ extract 31 11) src) #b000000000000000000000))
               (bvule #b10000000000 ((_ extract 10 0) src))))
      (a!4 (or (not (= ((_ extract 31 13) dst) #b0000000000000000000))
               (bvule #b1001110010100 ((_ extract 12 0) dst))))
      (a!5 (or (not (= ((_ extract 31 11) dst) #b000000000000000000000))
               (bvule #b11111010000 ((_ extract 10 0) dst))))
      (a!6 (bvadd N (bvmul #xffffffff (select32le (register state0) &a2))))
      (a!7 (not (= (select32le (register state0) &a2) #x00000001)))
      (a!8 (bvadd #x00000001
                  N
                  (bvmul #xffffffff (select32le (register state0) &a2))))
      (a!9 (select (ram state0)
      

Ok. Yes. Take 2, it is not obvious that checkeq over modified array is same as original array.

check(store(ram, addr), i) == check(ram, i) if addr >=  

In [ ]:
smt.ForAll([ram], ctx.store32(ram, src)

Graphical view of smt expr
Maybe fiddle with the program to exactly copy the assembly program (but even then, not _exact_ copy, since ignores details of sequencing, uninteresting state)
Better alias analysis on memory
GDB / debug output. full_simp

export lean version of verification conditions? Nah, that's just churn

```python
loop = kd.define(".loop", [live1,live2],
 [
  x := live1,
  y := live2,
  w := live3,
  smt.If(x,y,z)
][-1])
```

Make a DSL for describing blocks?
Use qbe?
Hmm am I reinventing MLCFG https://www.why3.org/doc/input_formats.html
https://coma.paulpatault.fr/x86-64.html micro x86? That's a new one to me

In a funny kind of way CPS bottom is a halfway house between functions returning and prolog predicate style.




In [6]:
import qbe_ir

ModuleNotFoundError: No module named 'qbe_ir'

In [149]:
import kdrag.printers.lean as lean

lean.of_expr(ps.vcs[0].vc(ctx))

'(not (((select32le (register (state0 : MemState!2)) (&a0 : (BitVec 32))) = (src : (BitVec 32))) && ((select32le (register (state0 : MemState!2)) (&a1 : (BitVec 32))) = (dst : (BitVec 32))) && (not ((not ((extract (src : (BitVec 32))) = 0#21)) || (bvule 1024#11 (extract (src : (BitVec 32)))))) && (not ((not ((extract (dst : (BitVec 32))) = 0#19)) || (bvule 5012#13 (extract (dst : (BitVec 32)))))) && ((not ((extract (dst : (BitVec 32))) = 0#21)) || (bvule 2000#11 (extract (dst : (BitVec 32))))) && ((select32le (register (state0 : MemState!2)) (&a2 : (BitVec 32))) = (N : (BitVec 32))) && ((N : (BitVec 32)) = 3#32) && ((resram : ((BitVec 32) -> (BitVec 8))) = (myloop (ram (state0 : MemState!2)) (N : (BitVec 32)))) && (not ((select32le (register (state0 : MemState!2)) (&a2 : (BitVec 32))) = 0#32))) || ((((src : (BitVec 32)) + (N : (BitVec 32))) = ((select32le (register (state0 : MemState!2)) (&a0 : (BitVec 32))) + (select32le (register (state0 : MemState!2)) (&a2 : (BitVec 32))))) && (((ds

In [150]:
import kdrag.contrib.pcode as pcode
import kdrag.contrib.pcode.asmspec as asmspec
from kdrag.all import *
import kdrag.theories.bitvec as bv
BV32 = smt.BitVecSort(32)
BV8 = smt.BitVecSort(8)


myloop = smt.Function("myloop", smt.ArraySort(BV32,BV8), BV32, smt.ArraySort(BV32,BV8))
i = smt.Const("i", BV32)
myloop = kd.define("myloop", [ram, i],
            smt.If(i == 0,
                ram,
                myloop(smt.Store(ram, dst + N - i, ram[src + N - i]), i-1)
            )
)

ctx = pcode.BinaryContext("/tmp/mycpy", langid="RISCV:LE:32:default")
a0 = ctx.state_vars["a0"]
a1 = ctx.state_vars["a1"]
a2 = ctx.state_vars["a2"]
ram = ctx.state_vars["ram"]
#write = ctx.state_vars["write"]

resram = smt.Array("resram", BV32, BV8) 

spec = asmspec.AsmSpec(ctx)
src,dst,N = smt.Consts("src dst N", BV32)
Nc = 3
spec.add_entry("mycpy", spec.find_label("mycpy"), smt.And(
    a0 == src,
    a1 == dst,
    #smt.Distinct(a0, a0 + 1, a0-1, a1),
    smt.ULT(src,1024),
    smt.ULT(dst, 5012),
    smt.UGE(dst, 2000),
    a2 == N,
    N == Nc,
    resram == myloop(ram, N)
    #smt.ULT(N, 2),
    ))
i = smt.Const("i", BV32)
spec.add_cut(".loop", spec.find_label(".loop"), smt.And(
    src + N == a0 + a2,
    dst + N == a1 + a2,
    smt.ULT(src,1024),
    smt.ULT(dst, 5012),
    smt.UGE(dst, 2000),
    resram == myloop(ram, a2),
    N == Nc
    #smt.ForAll([i], smt.ULT(0,i), smt.ULT(i, N-a2), ram[src + i] == ram[dst + i])
    ))

spec.add_exit(".done", spec.find_label(".done"), resram == ram)

ps = asmspec.AsmProofState(spec)
for i in range(len(ps.vcs)):
    print(i)
    ps.auto(i, by=[myloop.defn, ctx.select32.defn, ctx.store32.defn], timeout=5000)

Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 0) PCODE IMARK ram[400652:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 1) PCODE unique[580:4] = 0x0
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 2) PCODE unique[2880:4] = a0 + unique[580:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 3) PCODE unique[2900:1] = *[ram]unique[2880:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 4) PCODE t0 = sext(unique[2900:1])
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 0) PCODE IMARK ram[400650:2]
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 1) PCODE unique[10800:1] = a2 == 0x0
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 2) PCODE if (unique[10800:1]) goto ram[400662:4]
Executing SpecStmt: Cut(.loop, 0x400652, And(src + N == a0 + a2,
    dst + N == a1 + a2,
    ULT(src, 1024),
    ULT(dst, 5012),
    UGE(dst, 2000),
    resram == myloop(ram, a2),
    N == 3))
Executing 0x400656/4: sb t0,0x0(a1) at (4195926, 0) PCODE IMARK ram[400656:4]
Executing 0x400656/4: 

IndexError: tuple index out of range

In [157]:
l = ps.lemma(1)
l.unfold(ctx.select32, ctx.store32)
l.simp()
l.intros()
l.unfold(myloop)
#l.unfold(ctx.select32, ctx.store32)
l.simp()
"""
l.emt()
l.split(at=0)
l.top_goal().ctx
l.unfold(ctx.select32, ctx.store32)
l.simp()
l.top_goal().ctx
l.rw(-3)
"""
#l.auto(by=[ctx.select32.defn, ctx.store32.defn, myloop.defn])
l

?|= (let ((a!1 (concat (select (register state0) (bvadd #x00000003 &a0))
                   (select (register state0) (bvadd #x00000002 &a0))
                   (select (register state0) (bvadd #x00000001 &a0))
                   (select (register state0) &a0)))
      (a!2 (concat (select (register state0) (bvadd #x00000003 &a2))
                   (select (register state0) (bvadd #x00000002 &a2))
                   (select (register state0) (bvadd #x00000001 &a2))
                   (select (register state0) &a2)))
      (a!3 (concat (select (register state0) (bvadd #x00000003 &a1))
                   (select (register state0) (bvadd #x00000002 &a1))
                   (select (register state0) (bvadd #x00000001 &a1))
                   (select (register state0) &a1)))
      (a!4 (or (not (= ((_ extract 31 11) src) #b000000000000000000000))
               (bvule #b10000000000 ((_ extract 10 0) src))))
      (a!5 (or (not (= ((_ extract 31 13) dst) #b0000000000000000000))
             

In [ ]:
import kdrag.contrib.pcode as pcode
import kdrag.contrib.pcode.asmspec as asmspec
from kdrag.all import *
import kdrag.theories.bitvec as bv
BV32 = smt.BitVecSort(32)
BV8 = smt.BitVecSort(8)

ctx = pcode.BinaryContext("/tmp/mycpy", langid="RISCV:LE:32:default")
a0 = ctx.state_vars["a0"]
a1 = ctx.state_vars["a1"]
a2 = ctx.state_vars["a2"]
ram = ctx.state_vars["ram"]
#write = ctx.state_vars["write"]

Nc = 4
spec = asmspec.AsmSpec(ctx)
src,dst,N = smt.Consts("src dst N", BV32)
spec.add_entry("mycpy", spec.find_label("mycpy"), smt.And(
    a0 == src,
    a1 == dst,
    #smt.Distinct(a0, a0 + 1, a0-1, a1),
    smt.ULT(src,1024), #aliasing
    smt.ULT(dst, 5012),
    smt.UGE(dst, 2000),
    a2 == N,
    N == Nc,
    #smt.ULT(N, 2),
    ))
i = smt.Const("i", BV32)
spec.add_cut(".loop", spec.find_label(".loop"), smt.And(
    src + N == a0 + a2,
    dst + N == a1 + a2,
    N == Nc,
    smt.ULT(src,1024), #aliasing
    smt.ULT(dst, 5012),
    smt.UGE(dst, 2000),
    #ram[src + N - a2] == ram[dst + N - a2],
    #smt.ForAll([i], smt.ULE(0,i), smt.ULT(i, N-a2), ram[src + i] == ram[dst + i])
    checkeq(ram, src, dst, N)
    ))

spec.add_exit(".done", spec.find_label(".done"), smt.ForAll([i], smt.ULE(0,i), smt.ULT(i, N), ram[src + i] == ram[dst + i]))
#smt.And(ram[src] == ram[dst], ram[src+N-1] == ram[dst+N-1]))

ps = asmspec.AsmProofState(spec)
#for i in range(len(ps.vcs)):
#    ps.auto(i)
#ps
l = ps.lemma(0)
l.auto()
l.qed()

l = ps.lemma(1)
l.intros()
_i = l.fix()
l.split(at=0)
l.auto(by=[f.defn for f in ctx.definitions])
l.qed()

#l.auto(by=[f.defn for f in ctx.definitions])
#l.vc.verify(ctx)
#ps
#l = ps.lemma(0)
#l.intros()


Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 0) PCODE IMARK ram[400652:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 1) PCODE unique[580:4] = 0x0
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 2) PCODE unique[2880:4] = a0 + unique[580:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 3) PCODE unique[2900:1] = *[ram]unique[2880:4]
Executing 0x400652/4: lb t0,0x0(a0) at (4195922, 4) PCODE t0 = sext(unique[2900:1])
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 0) PCODE IMARK ram[400650:2]
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 1) PCODE unique[10800:1] = a2 == 0x0
Executing 0x400650/2: c.beqz a2,0x400662 at (4195920, 2) PCODE if (unique[10800:1]) goto ram[400662:4]
Executing SpecStmt: Cut(.loop, 0x400652, And(src + N == a0 + a2,
    dst + N == a1 + a2,
    N == 4,
    ULT(src, 1024),
    ULT(dst, 5012),
    UGE(dst, 2000),
    ForAll(i,
           Implies(And(ULE(0, i), ULT(i, N - a2)),
                   ram[src + i] == ram[dst + i]))))
Executing 0x400

ValueError: fixes tactic failed. Not a forall And(src + N ==
    select32le(register(state0), &a0) +
    select32le(register(state0), &a2),
    dst + N ==
    select32le(register(state0), &a1) +
    select32le(register(state0), &a2),
    N == 4,
    Not(Or(Not(Extract(31, 11, src) == 0),
           ULE(1024, Extract(10, 0, src)))),
    Not(Or(Not(Extract(31, 13, dst) == 0),
           ULE(5012, Extract(12, 0, dst)))),
    Or(Not(Extract(31, 11, dst) == 0),
       ULE(2000, Extract(10, 0, dst))),
    ForAll(i,
           Or(Store(ram(state0),
                    select32le(register(state0), &a1),
                    store32le(register(state0),
                              &t0,
                              Concat(Extract(7,
                                        7,
                                        ram(state0)[select32le(register(state0),
                                        &a0)]),
                                     Extract(7,
                                        7,
   

In [43]:

l = ps.lemma(2)
l.intros()
l.split(at=0)
l.split()
l.auto()
l.auto()
l.auto()
l.auto()
l.auto()
l.auto()
_i = l.fix()
l.top_goal().ctx
#print(l.top_goal().goal.sexpr())
l.specialize(-2, _i)
l.specialize(-3, _i-1)
l.specialize(-4, _i-2)
l.specialize(-5, _i+1)
l.clear(-6)
l.emt()
#l.auto(by=[f.defn for f in ctx.definitions])



ValueError: ('EMT failed. Goal is already in simplest form.', Or(Store(ram(state0),
         select32le(register(state0), &a1),
         store32le(register(state0),
                   &t0,
                   Concat(Extract(7,
                                  7,
                                  ram(state0)[select32le(register(state0),
                                        &a0)]),
                          Extract(7,
                                  7,
                                  ram(state0)[select32le(register(state0),
                                        &a0)]),
                          Extract(7,
                                  7,
                                  ram(state0)[select32le(register(state0),
                                        &a0)]),
                          Extract(7,
                                  7,
                                  ram(state0)[select32le(register(state0),
                                        &a0)]),
                       

In [12]:
l.top_goal().ctx

[src + N ==
 select32le(register(state0), &a0) +
 select32le(register(state0), &a2),
 dst + N ==
 select32le(register(state0), &a1) +
 select32le(register(state0), &a2),
 N == 4,
 Not(Or(Not(Extract(31, 11, src) == 0),
        ULE(1024, Extract(10, 0, src)))),
 Not(Or(Not(Extract(31, 13, dst) == 0),
        ULE(5012, Extract(12, 0, dst)))),
 Or(Not(Extract(31, 11, dst) == 0),
    ULE(2000, Extract(10, 0, dst))),
 ForAll(i,
        Or(ram(state0)[src + i] == ram(state0)[dst + i],
           ULE(N +
               4294967295*select32le(register(state0), &a2),
               i))),
 Not(select32le(register(state0), &a2) == 1)]

In [16]:
print(l.top_goal().goal.sexpr())

(let ((a!1 (= (bvadd src N)
              (bvadd (select32le (register state0) &a0)
                     (select32le (register state0) &a2))))
      (a!2 (= (bvadd dst N)
              (bvadd (select32le (register state0) &a1)
                     (select32le (register state0) &a2))))
      (a!3 (or (not (= ((_ extract 31 11) src) #b000000000000000000000))
               (bvule #b10000000000 ((_ extract 10 0) src))))
      (a!4 (or (not (= ((_ extract 31 13) dst) #b0000000000000000000))
               (bvule #b1001110010100 ((_ extract 12 0) dst))))
      (a!5 (or (not (= ((_ extract 31 11) dst) #b000000000000000000000))
               (bvule #b11111010000 ((_ extract 10 0) dst))))
      (a!6 (forall ((i (_ BitVec 32)))
             (let ((a!1 ((_ extract 7 7)
                          (select (ram state0)
                                  (select32le (register state0) &a0))))
                   (a!4 (bvadd #x00000001
                               N
                               (bvm

In [ ]:
def NoOverlap(x, Nx, y, Ny) -> smt.BoolRef:
    """
    No overlap between [x, x+Nx) and [y, y+Ny)
    """
    x1 = smt.ZeroExt(1, x)
    y1 = smt.ZeroExt(1, y)
    #return smt.Or(
    #           smt.And(smt.ULT(x, y), smt.ULT(x1, x1 + Nx), smt.ULE(x + Nx, y1),
    #            smt.And(smt.ULT(x,y),  smt.ULT(x1 + Nx, x),  )
    #            smt.And(smt.ULE(y1 + Ny, x1), smt.ULT(y, x)))

    #return smt.Or(smt.And(smt.ULE(x + Nx, y), smt.ULT(x, y), smt.BVAddNoOverflow(x, Nx, False)), 
    #              smt.And(smt.ULE(y + Ny, x), smt.ULT(y, x), smt.BVAddNoOverflow(y, Ny, False)))

def NoOverlap2(x, Nx, y, Ny) -> smt.BoolRef:
    """
    No overlap between [x, x+Nx) and [y, y+Ny)
    """
    return smt.Distinct([x + i for i in range(Nx)] + [y + j for j in range(Ny)])

def NoOverlap3(x, Nx, y, Ny) -> smt.BoolRef:
    """
    No overlap between [x, x+Nx) and [y, y+Ny)
    """
    i, j = smt.Ints('i j')
    return smt.ForAll([i, j], i >= 0, i < Nx, j >= 0, j < Ny, x + i != y + j)

x,y = smt.BitVecs('x y', 16)
for Nx in [1,2,3,4]:
    for Ny in [1,2,3,4]:
        print(f"Proving NoOverlap with Nx={Nx} Ny={Ny}")
        kd.prove(NoOverlap(x, Nx, y, Ny) == NoOverlap2(x, Nx, y, Ny))

# 6502
https://github.com/NationalSecurityAgency/ghidra/issues/5275 noah's decompiler problem

https://dramforever.github.io/easyriscv/
https://github.com/Arlet/verilog-6502
http://www.aholme.co.uk/6502/Main.htm - hmm. Maybe open enough to go to analog layer?

https://8bitworkshop.com/
https://www.dabeaz.com/superboard/asm6502.py

https://www.masswerk.at/6502/assembler.html
https://www.masswerk.at/6502/6502_instruction_set.html

https://famicom.party/book/
cc65 https://en.wikipedia.org/wiki/Cc65


In [172]:
import pypcode
ctx = pypcode.Context(language="6502:LE:16:default")
ctx.translate(bytes.fromhex("6942")).ops

[IMARK RAM[0:2],
 unique[1a80:1] = 0x42,
 unique[3580:1] = unique[1a80:1],
 unique[3600:1] = C,
 C = carry(A, unique[3580:1]),
 unique[3700:1] = A + unique[3580:1],
 A = unique[3700:1] + unique[3600:1],
 Z = A == 0x0,
 N = A s< 0x0,
 V = C]

# avr
https://github.com/m3y54m/start-avr
```
sudo apt-get install gcc build-essential
sudo apt-get install gcc-avr binutils-avr avr-libc gdb-avr
```

In [173]:
%%file /tmp/testavr.S

   add r16, r17
   ret

Writing /tmp/testavr.S


In [176]:
! avr-gcc -mmcu=atmega328p -o /tmp/testavr /tmp/testavr.S && avr-objdump -d /tmp/testavr

/usr/lib/gcc/avr/7.3.0/../../../avr/bin/ld: cannot find crtatmega328p.o: No such file or directory
/usr/lib/gcc/avr/7.3.0/../../../avr/bin/ld: cannot find -lm
/usr/lib/gcc/avr/7.3.0/../../../avr/bin/ld: cannot find -lc
/usr/lib/gcc/avr/7.3.0/../../../avr/bin/ld: cannot find -latmega328p
collect2: error: ld returned 1 exit status


# icicle
shell out to qemu with print all instructions and registers flag.
Build small stub to setup countermodel



In [1]:
import icicle

icicle.architectures()

['i686', 'x86_64', 'aarch64']

In [ ]:
import unicorn
unicorn 